# **Source Code for HYDRA with across all models.**

**HYDRA** Model

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from itertools import product
from transformers import RobertaTokenizer, RobertaModel
import os
from collections import Counter

# Set reproducibility seed
np.random.seed(42)  # NumPy seed
torch.manual_seed(42)  # PyTorch seed (for CUDA, add torch.cuda.manual_seed_all(42) if using GPU)

# VAE Model (Base Case)
class VAE(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        # Encoder
        self.encoder_layer = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2)
        )

        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(128, 64)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, output_dim)
        )

    def encode(self, x):
        x = self.encoder_layer(x)
        mu = self.fc1(x)
        log_var = self.fc2(x)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std) * 0.001
        z = mu + eps * std
        return z

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconstructed = self.decode(z)
        return x_reconstructed, mu, log_var

def loss_function(recon_x, x, mu, log_var):
    MSE = F.mse_loss(recon_x, x, reduction='mean')
    return MSE

# GraphCodeBERT Feature Extraction
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base")

def get_code_features(code):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Heuristic Detection Functions
def detect_missing_malloc_check(code: str) -> int:
    alloc_pattern = r'\b(\w+)\s*\*\s*(\w+)\s*=\s*\([^\)]*\)\s*(malloc|calloc|realloc)\s*\([^;]+?\);|\b(\w+)\s*\*\s*(\w+)\s*=\s*(malloc|calloc|realloc)\s*\([^;]+?\);'
    matches = re.findall(alloc_pattern, code)
    ptrs = [match[1] if match[1] else match[4] for match in matches if match[1] or match[4]]
    for ptr in ptrs:
        is_checked = False
        is_used = False
        null_check_patterns = [rf'if\s*\(\s*{ptr}\s*\)', rf'if\s*\(\s*{ptr}\s*!=\s*NULL\s*\)', rf'if\s*\(\s*!{ptr}\s*\)', rf'if\s*\(\s*NULL\s*!=\s*{ptr}\s*\)']
        usage_patterns = [rf'{ptr}\s*\[', rf'\*\s*{ptr}', rf'{ptr}\s*->', rf'{ptr}\s*\(', rf'{ptr}\s*=']
        for pat in null_check_patterns:
            if re.search(pat, code):
                is_checked = True
                break
        for pat in usage_patterns:
            if re.search(pat, code):
                is_used = True
                break
        if is_used and not is_checked:
            return 1
    return 0

def missing_null_check_func(code):
    functions = re.findall(r'\b\w+\s+\w+\s*\([^)]*\)\s*\{[^{}]*\}', code, re.DOTALL)
    for func in functions:
        ptr_decl = re.findall(r'\b\w+\s*\*\s*(\w+)\s*=\s*[^;]+;', func)
        ptr_func_decl = re.findall(r'\b\w+\s*\*\s*(\w+)\s*;', func)
        ptr_vars = list(set(ptr_decl + ptr_func_decl))
        for ptr in ptr_vars:
            unsafe_use = re.search(rf'[^a-zA-Z0-9_]({ptr})(->|\[\d*]|[^\w]?\*)|\b{ptr}\s*\(', func)
            if not unsafe_use:
                continue
            null_check = re.search(rf'if\s*\(\s*(!\s*{ptr}|{ptr}\s*==\s*NULL|{ptr}\s*!=\s*NULL)\s*\)', func)
            if not null_check:
                return 1
    return 0

def detect_race_condition(code: str) -> bool:
    field_assignment_pattern = r'\b\w+\s*(->|\.)\s*\w+\s*=.*?;'
    control_block_pattern = r'\b(if|while|for|switch)\s*\([^\)]+\)\s*{[^}]*' + field_assignment_pattern + r'[^}]*}'
    matches_control_blocks = re.findall(control_block_pattern, code, re.DOTALL)
    locking_primitive_pattern = r'\b(mutex_lock|pthread_mutex_lock|spin_lock)\b'
    unlocking_primitive_pattern = r'\b(mutex_unlock|pthread_mutex_unlock|spin_unlock)\b'
    has_locking = re.search(locking_primitive_pattern, code)
    has_unlocking = re.search(unlocking_primitive_pattern, code)
    if matches_control_blocks and not (has_locking and has_unlocking):
        return True
    return False

def logging_but_no_blocking(code):
    log_lines = re.findall(r'(syslog\([^)]*\)|printk\([^)]*\))', code)
    for line in log_lines:
        log_index = code.find(line)
        snippet = code[log_index:log_index+150]
        if not re.search(r'(return|exit|break|continue)', snippet):
            return 1
    return 0

def split_into_functions(code):
    functions = re.findall(r'([a-zA-Z_][a-zA-Z0-9_\*\s]*\s+[a-zA-Z_][a-zA-Z0-9_]*\s*\(.*?\)\s*\{.*?\})', code, re.DOTALL)
    return functions

def missing_bounds_check(code):
    functions = split_into_functions(code)
    risky_keywords = ['recv', 'read', 'strcpy', 'memcpy', 'gets', 'strcat', 'write']
    for func in functions:
        found_risky = any(kw in func for kw in risky_keywords)
        has_check = any('if' in line and ('<' in line or '>' in line or '<=' in line or '>=' in line) for line in func.splitlines())
        if found_risky and not has_check:
            return 1
    return 0

def analyze_risks(code):
    risk_flags = {
        "Missing Null Check": missing_null_check_func(code),
        "Race Condition": detect_race_condition(code),
        "Missing Bounds Check": missing_bounds_check(code),
        "Unsafe Memory Allocation": detect_missing_malloc_check(code),
        "Logging Without Halting": logging_but_no_blocking(code),
        "issue_detected": 0
    }
    if any(v != 0 for v in risk_flags.values() if v is not False):
        risk_flags["issue_detected"] = 1
    return risk_flags

# Load and preprocess data
train_path = '/home/phoenix/Desktop/TA/Re_ New Code/Linux-Data-Updated.csv'
test_paths = {
    'Android': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Android-Data.csv',
    'Chrome': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Chrome-Data.csv',
    'ImageMagick': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/ImageMagick-Data.csv'
}

def load_or_generate_embeddings(df, path, filename):
    embed_file = f"{filename}_embeddings.npy"
    heur_file = f"{filename}_heuristics.npy"
    commit_file = f"{filename}_commit_ids.npy"

    if os.path.exists(embed_file) and os.path.exists(heur_file) and os.path.exists(commit_file):
        embeddings = np.load(embed_file)
        heuristics = np.load(heur_file)
        commit_ids = np.load(commit_file)
    else:
        embeddings = []
        heuristics = []
        commit_ids = []
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing GraphCodeBERT embeddings and heuristics ({filename})"):
            code = row['PCode']
            embedding = get_code_features(code)
            risk_flags = analyze_risks(code)
            embeddings.append(embedding)
            heuristics.append([risk_flags[h] for h in [
                "Missing Null Check", "Race Condition", "Missing Bounds Check",
                "Unsafe Memory Allocation", "Logging Without Halting", "issue_detected"
            ]])
            commit_ids.append(row['Commit_Id'])
        embeddings = np.array(embeddings)
        heuristics = np.array(heuristics)
        np.save(embed_file, embeddings)
        np.save(heur_file, heuristics)
        np.save(commit_file, commit_ids)
    return embeddings, heuristics, commit_ids

df_train = pd.read_csv(train_path)
df_train['PCode'] = df_train['PCode'].astype(str)
embeddings_train, heuristics_train, commit_ids_train = load_or_generate_embeddings(df_train, train_path, "train")

# Combine embeddings and heuristics for training and split for validation
features_train = np.concatenate([embeddings_train, heuristics_train], axis=1)
train_data, val_data = train_test_split(features_train, test_size=0.2, random_state=42)

# Hyperparameter tuning for latent_dim and n_clusters
input_dim = features_train.shape[1]  # 774 (768 + 6)
output_dim = input_dim
param_grid = {
    'latent_dim': [5, 10, 20],
    'n_clusters': [2, 3, 5]
}
results = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for params in tqdm(product(*param_grid.values()), desc="Tuning VAE and K-means"):
    latent_dim, n_clusters = params
    vae = VAE(input_dim, output_dim).to(device)
    optimizer = optim.Adam(vae.parameters(), lr=1e-3)
    criterion = loss_function
    vae.train()
    best_val_loss = float('inf')
    patience = 20
    trigger_times = 0
    for epoch in range(200):
        vae.train()
        total_loss = 0
        total_recon_loss = 0
        for i in range(0, len(train_data), 128):
            batch = torch.FloatTensor(train_data[i:i+128]).to(device)
            optimizer.zero_grad()
            recon_batch, mu, log_var = vae(batch)
            recon_loss = criterion(recon_batch, batch, mu, log_var)
            recon_loss.backward()
            optimizer.step()
            total_loss += recon_loss.item()
            total_recon_loss += recon_loss.item()
        # Validation phase
        vae.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, len(val_data), 128):
                val_batch = torch.FloatTensor(val_data[i:i+128]).to(device)
                recon_val_batch, _, _ = vae(val_batch)
                val_loss += criterion(recon_val_batch, val_batch, mu, log_var).item()
        avg_val_loss = val_loss / (len(val_data) / 128)
        if (epoch + 1) % 10 == 0:
            avg_recon_loss = total_recon_loss / (len(train_data) / 128)
            print(f"Epoch [{epoch+1}/200], Latent Dim={latent_dim}, Train Recon Loss: {avg_recon_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            trigger_times = 0
        else:
            trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at Epoch {epoch+1} for latent_dim={latent_dim}, n_clusters={n_clusters}")
            break
    vae.eval()
    with torch.no_grad():
        _, mu, log_var = vae(torch.FloatTensor(train_data).to(device))
        z_train = vae.reparameterize(mu, log_var)
    z_train = z_train.cpu().numpy()
    clustering = KMeans(n_clusters=n_clusters, random_state=42)
    train_clusters = clustering.fit_predict(z_train)
    if len(np.unique(train_clusters)) > 1:
        silhouette = silhouette_score(z_train, train_clusters)
        results.append({'latent_dim': latent_dim, 'n_clusters': n_clusters, 'silhouette': silhouette})
        print(f"latent_dim={latent_dim}, n_clusters={n_clusters}, Silhouette Score: {silhouette:.4f}")
best_result = max(results, key=lambda x: x['silhouette'])
print(f"Best Configuration: latent_dim={best_result['latent_dim']}, n_clusters={best_result['n_clusters']}, Silhouette Score: {best_result['silhouette']:.4f}")
best_params = {'latent_dim': best_result['latent_dim'], 'n_clusters': best_result['n_clusters']}

# Train final VAE with best parameters
vae = VAE(input_dim, output_dim).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
criterion = loss_function
vae.train()
best_val_loss = float('inf')
patience = 20
trigger_times = 0
train_losses = []
val_losses = []
for epoch in range(200):
    vae.train()
    total_loss = 0
    total_recon_loss = 0
    for i in range(0, len(train_data), 128):
        batch = torch.FloatTensor(train_data[i:i+128]).to(device)
        optimizer.zero_grad()
        recon_batch, mu, log_var = vae(batch)
        recon_loss = criterion(recon_batch, batch, mu, log_var)
        recon_loss.backward()
        optimizer.step()
        total_loss += recon_loss.item()
        total_recon_loss += recon_loss.item()
    # Validation phase
    vae.eval()
    val_loss = 0
    with torch.no_grad():
        for i in range(0, len(val_data), 128):
            val_batch = torch.FloatTensor(val_data[i:i+128]).to(device)
            recon_val_batch, _, _ = vae(val_batch)
            val_loss += criterion(recon_val_batch, val_batch, mu, log_var).item()
    avg_val_loss = val_loss / (len(val_data) / 128)
    avg_train_loss = total_loss / (len(train_data) / 128)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/200], Train Recon Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
    if trigger_times >= patience:
        print(f"Early stopping at Epoch {epoch+1}")
        break
vae.eval()
with torch.no_grad():
    _, mu, log_var = vae(torch.FloatTensor(train_data).to(device))
    z_train = vae.reparameterize(mu, log_var)
z_train = z_train.cpu().numpy()

# Process test datasets with initial embeddings and heuristics
test_data = {}
for name, path in test_paths.items():
    df_test = pd.read_csv(path)
    df_test['PCode'] = df_test['PCode'].astype(str)
    embeddings_test, heuristics_test, commit_ids_test = load_or_generate_embeddings(df_test, path, name)
    features_test = np.concatenate([embeddings_test, heuristics_test], axis=1)
    test_data[name] = {
        'features': features_test,
        'heuristics': heuristics_test,
        'commit_ids': np.array(commit_ids_test),
        'codes': df_test['PCode'].values
    }

# Recompute recon errors for test data
def compute_reconstruction_error(model, data, device, batch_size=128):
    model.eval()
    recon_errors = []
    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = torch.FloatTensor(data[i:i+batch_size]).to(device)
            recon_batch, _, _ = model(batch)
            error = F.mse_loss(recon_batch, batch, reduction='none').mean(dim=1)
            recon_errors.append(error.cpu().numpy())
    return np.concatenate(recon_errors)


In [ ]:
# Apply VAE + clustering to test datasets and analyze
heuristic_names = [
    "Missing Null Check", "Race Condition", "Missing Bounds Check",
    "Unsafe Memory Allocation", "Logging Without Halting", "None"
]
for name in test_data:
    features = test_data[name]['features']
    heuristics = test_data[name]['heuristics']
    commit_ids = test_data[name]['commit_ids']
    codes = test_data[name]['codes']

    with torch.no_grad():
        _, mu, log_var = vae(torch.FloatTensor(features).to(device))
        z_test = vae.reparameterize(mu, log_var)
    z_test = z_test.cpu().numpy()

    clustering = AgglomerativeClustering(n_clusters=best_params['n_clusters'], linkage='ward')
    clusters = clustering.fit_predict(z_test)

    recon_error = compute_reconstruction_error(vae, features, device)

    # Save to dictionary
    test_data[name].update({
        'latent_z': z_test,
        'clusters': clusters,
        'recon_error': recon_error
    })

    # Cluster evaluation metrics
    silhouette = silhouette_score(z_test, clusters)
    ch_score = calinski_harabasz_score(z_test, clusters)
    db_score = davies_bouldin_score(z_test, clusters)
    print(f"\n[{name}] Clustering Metrics:")
    print(f"Silhouette Score: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

    # Top anomalies by recon error
    print(f"Top 5 Anomalous Functions in {name}:")
    top_idx = np.argsort(-recon_error)[:5]
    for idx in top_idx:
        print(f"Commit ID: {commit_ids[idx]}, Error: {recon_error[idx]:.4f}\n{codes[idx][:200]}...\n")

    # Save output dataframe
    df_out = pd.DataFrame({
        'Commit_ID': commit_ids,
        'Cluster': clusters,
        'Recon_Error': recon_error,
        'Heuristic_Label': [heuristic_names[np.argmax(h) if np.max(h) > 0 else -1] for h in heuristics],
        'Code': codes
    })
    df_out.to_csv(f"{name}_cluster_results.csv", index=False)

    # Cluster summary
    print(f"Heuristic distribution by cluster in {name}:")
    cluster_heuristic_counts = []
    for c in range(best_params['n_clusters']):
        idx = np.where(clusters == c)[0]
        hsum = np.sum(heuristics[idx], axis=0)
        hsum[-1] = np.sum(heuristics[idx, -1] == 0)  # Correct 'None' count
        cluster_heuristic_counts.append(hsum)
        top = np.argmax(hsum[:-1])  # exclude 'None'
        print(f"Cluster {c}: Dominant Heuristic = {heuristic_names[top]}, Count = {hsum[top]}")

    # t-SNE plot with cluster markers, boundaries, and side labels
    tsne = TSNE(n_components=2, random_state=42)
    tsne_coords = tsne.fit_transform(z_test)
    tsne_df = pd.DataFrame(tsne_coords, columns=['TSNE1', 'TSNE2'])
    tsne_df['Cluster'] = clusters
    tsne_df['Heuristic'] = [heuristic_names[np.argmax(h) if np.max(h) > 0 else -1] for h in heuristics]
    tsne_df['ReconError'] = recon_error
    cluster_df = pd.DataFrame(cluster_heuristic_counts, columns=heuristic_names)

    plt.figure(figsize=(14, 8))
    scatter = sns.scatterplot(data=tsne_df, x='TSNE1', y='TSNE2', hue='Heuristic', style='Cluster',
                              palette={
                                  'Missing Null Check': 'green',
                                  'Race Condition': 'red',
                                  'Missing Bounds Check': 'blue',
                                  'Unsafe Memory Allocation': 'purple',
                                  'Logging Without Halting': 'orange',
                                  'None': 'gray'
                              }, marker='o', s=100)
    plt.title(f"t-SNE of {name} - Regex + GraphCodeBERT + VAE + Kmeans (HYDRA)")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend(title=' ', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Compute and plot convex hulls for cluster boundaries
    from scipy.spatial import ConvexHull
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 1:  # Need at least 2 points for a hull
            cluster_points = tsne_coords[cluster_indices]
            hull = ConvexHull(cluster_points)
            for simplex in hull.simplices:
                plt.plot(cluster_points[simplex, 0], cluster_points[simplex, 1], 'k-', alpha=0.5)

    # Label clusters outside the boundaries
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 0:
            cluster_points = tsne_coords[cluster_indices]
            cluster_center = cluster_points.mean(axis=0)
            x_range = tsne_coords[:, 0].max() - tsne_coords[:, 0].min()
            y_range = tsne_coords[:, 1].max() - tsne_coords[:, 1].min()
            offset = 0.1 * max(x_range, y_range)
            outward_x = cluster_center[0] + offset * np.sign(x_range)
            outward_y = cluster_center[1] + offset * np.sign(y_range)
            plt.text(outward_x, outward_y, f'Cluster {cluster}',
                     fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))


    plt.tight_layout()
    plt.savefig(f"{name}_tsne_plot.png")
    plt.show()

    # Count heuristics per cluster
    cluster_heuristics = {i: {h: 0 for h in heuristic_names} for i in range(best_params['n_clusters'])}
    for idx, cluster in enumerate(clusters):
        for h_idx, h_name in enumerate(heuristic_names):
            if heuristics[idx, h_idx] == 1:
                cluster_heuristics[cluster][h_name] += 1

    # Print total number of each heuristic per cluster
    print(f"\n{name} Dataset - Cluster Heuristic Counts:")
    for cluster in range(best_params['n_clusters']):
        print(f"\nCluster {cluster}:")
        for h_name, count in cluster_heuristics[cluster].items():
            print(f"{h_name}: {count}")

    # Detect and print false positives for "None" heuristic
    print(f"\n{name} Dataset - False Positives for 'None' Heuristic:")
    false_positives = []
    for idx in range(len(heuristics)):
        if heuristics[idx, -1] == 1:  # Sample labeled as "None"
            if np.any(heuristics[idx, :-1] == 1):  # Check if any other heuristic is true
                false_positive_heuristics = [h for h_idx, h in enumerate(heuristic_names[:-1]) if heuristics[idx, h_idx] == 1]
                false_positives.append((idx, false_positive_heuristics))

    if false_positives:
        print(f"Found {len(false_positives)} false positive(s) where 'None' was incorrectly assigned:")
        for idx, conflicting_heuristics in false_positives:
            print(f"\nCommit ID: {commit_ids[idx]}")
            print(f"Conflicting Heuristics: {', '.join(conflicting_heuristics)}")
            print(f"Code Snippet: {codes[idx][:200]}...\n")
    else:
        print("No false positives found for 'None' heuristic.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler


# Dataset and metric setup
datasets = list(test_data.keys())
metrics = ['Silhouette', 'CH', 'DB']
x_locs = np.arange(len(metrics))
bar_width = 0.2
colors = ['tab:blue', 'tab:orange', 'tab:green']

# Step 1: Collect raw scores
raw_scores = []
for name in datasets:
    sil = silhouette_score(test_data[name]['latent_z'], test_data[name]['clusters'])
    ch = calinski_harabasz_score(test_data[name]['latent_z'], test_data[name]['clusters']) / 1000
    db = davies_bouldin_score(test_data[name]['latent_z'], test_data[name]['clusters'])
    raw_scores.append([sil, ch, db])

raw_scores = np.array(raw_scores)  # shape: (num_datasets, num_metrics)

# Step 2: Normalize for visualization
scaler = MinMaxScaler()
normalized_scores = scaler.fit_transform(raw_scores)

# Add offset to avoid invisible bars
normalized_scores += 0.05
normalized_scores = np.clip(normalized_scores, 0, 1)

# Step 3: Plotting
fig, ax = plt.subplots(figsize=(10, 6))

for i, (name, color) in enumerate(zip(datasets, colors)):
    offset = (i - 1) * bar_width
    bars = ax.bar(x_locs + offset, normalized_scores[i], width=bar_width, label=name, color=color)

    # Add actual score labels above each bar
    for j, bar in enumerate(bars):
        height = bar.get_height()
        actual_score = raw_scores[i][j]
        ax.text(bar.get_x() + bar.get_width() / 2, height + 0.02,
                f"{actual_score:.2f}", ha='center', va='bottom', fontsize=9)

# Step 4: Axes formatting
ax.set_ylabel("Normalized Metric Score (0–1)\n Actual Metric Values Printed Above Each Bar", fontsize=12)
ax.set_xticks(x_locs)
ax.set_xticklabels(metrics, fontsize=12)
ax.set_title("Normalized Clustering Metrics Across Test Datasets (Android, Chrome, IMageMagick) for HYDRA", fontsize=14)
ax.grid(True, axis='y', linestyle='--', alpha=0.4)
ax.legend(title="Datasets", fontsize=10)

plt.tight_layout()
plt.savefig("clustering_metrics_improved.png", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.ndimage import gaussian_filter1d

# Ensure train/val losses are NumPy arrays
train_losses = np.array(train_losses)
val_losses = np.array(val_losses)

# Smooth only if sufficient points exist
sigma = 1 if len(train_losses) >= 10 else 0
train_losses_smooth = gaussian_filter1d(train_losses, sigma=sigma)
val_losses_smooth = gaussian_filter1d(val_losses, sigma=sigma)

# Best epoch (min validation loss)
best_epoch = np.argmin(val_losses)
best_loss = val_losses[best_epoch]

# Plot style
sns.set(style="whitegrid", context='talk')

plt.figure(figsize=(12, 7))
plt.plot(train_losses_smooth, label='Train Loss', color='steelblue', linewidth=2.5)
plt.plot(val_losses_smooth, label='Validation Loss', color='darkorange', linewidth=2.5)

# Best epoch marker
plt.scatter(best_epoch, best_loss, color='red', s=100, zorder=5, label=f'Best Epoch ({best_epoch})')

# Annotation near best point
y_offset = (val_losses.max() - val_losses.min()) * 0.05
plt.annotate(f'{best_loss:.2f}',
             xy=(best_epoch, best_loss),
             xytext=(best_epoch + 3, best_loss + y_offset),
             arrowprops=dict(arrowstyle='->', lw=1.5, color='gray'),
             fontsize=12, bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='gray', alpha=0.8))

# Labels and title
plt.title('VAE Loss Curve During Training (HYDRA)', fontsize=18)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)

plt.legend(fontsize=12, loc='upper right')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('vae_loss_curve_a_star.png', dpi=300, bbox_inches='tight')
plt.show()







**M3** Model

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from itertools import product
from transformers import RobertaTokenizer, RobertaModel
import os
from collections import Counter
from scipy.spatial import ConvexHull

# Set reproducibility seed
np.random.seed(42)  # NumPy seed
torch.manual_seed(42)  # PyTorch seed

# GraphCodeBERT Feature Extraction
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base")

def get_code_features(code):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Heuristic Detection Functions
def detect_missing_malloc_check(code: str) -> int:
    alloc_pattern = r'\b(\w+)\s*\*\s*(\w+)\s*=\s*\([^\)]*\)\s*(malloc|calloc|realloc)\s*\([^;]+?\);|\b(\w+)\s*\*\s*(\w+)\s*=\s*(malloc|calloc|realloc)\s*\([^;]+?\);'
    matches = re.findall(alloc_pattern, code)
    ptrs = [match[1] if match[1] else match[4] for match in matches if match[1] or match[4]]
    for ptr in ptrs:
        is_checked = False
        is_used = False
        null_check_patterns = [rf'if\s*\(\s*{ptr}\s*\)', rf'if\s*\(\s*{ptr}\s*!=\s*NULL\s*\)', rf'if\s*\(\s*!{ptr}\s*\)', rf'if\s*\(\s*NULL\s*!=\s*{ptr}\s*\)']
        usage_patterns = [rf'{ptr}\s*\[', rf'\*\s*{ptr}', rf'{ptr}\s*->', rf'{ptr}\s*\(', rf'{ptr}\s*=']
        for pat in null_check_patterns:
            if re.search(pat, code):
                is_checked = True
                break
        for pat in usage_patterns:
            if re.search(pat, code):
                is_used = True
                break
        if is_used and not is_checked:
            return 1
    return 0

def missing_null_check_func(code):
    functions = re.findall(r'\b\w+\s+\w+\s*\([^)]*\)\s*\{[^{}]*\}', code, re.DOTALL)
    for func in functions:
        ptr_decl = re.findall(r'\b\w+\s*\*\s*(\w+)\s*=\s*[^;]+;', func)
        ptr_func_decl = re.findall(r'\b\w+\s*\*\s*(\w+)\s*;', func)
        ptr_vars = list(set(ptr_decl + ptr_func_decl))
        for ptr in ptr_vars:
            unsafe_use = re.search(rf'[^a-zA-Z0-9_]({ptr})(->|\[\d*]|[^\w]?\*)|\b{ptr}\s*\(', func)
            if not unsafe_use:
                continue
            null_check = re.search(rf'if\s*\(\s*(!\s*{ptr}|{ptr}\s*==\s*NULL|{ptr}\s*!=\s*NULL)\s*\)', func)
            if not null_check:
                return 1
    return 0

def detect_race_condition(code: str) -> bool:
    field_assignment_pattern = r'\b\w+\s*(->|\.)\s*\w+\s*=.*?;'
    control_block_pattern = r'\b(if|while|for|switch)\s*\([^\)]+\)\s*{[^}]*' + field_assignment_pattern + r'[^}]*}'
    matches_control_blocks = re.findall(control_block_pattern, code, re.DOTALL)
    locking_primitive_pattern = r'\b(mutex_lock|pthread_mutex_lock|spin_lock)\b'
    unlocking_primitive_pattern = r'\b(mutex_unlock|pthread_mutex_unlock|spin_unlock)\b'
    has_locking = re.search(locking_primitive_pattern, code)
    has_unlocking = re.search(unlocking_primitive_pattern, code)
    if matches_control_blocks and not (has_locking and has_unlocking):
        return True
    return False

def logging_but_no_blocking(code):
    log_lines = re.findall(r'(syslog\([^)]*\)|printk\([^)]*\))', code)
    for line in log_lines:
        log_index = code.find(line)
        snippet = code[log_index:log_index+150]
        if not re.search(r'(return|exit|break|continue)', snippet):
            return 1
    return 0

def split_into_functions(code):
    functions = re.findall(r'([a-zA-Z_][a-zA-Z0-9_\*\s]*\s+[a-zA-Z_][a-zA-Z0-9_]*\s*\(.*?\)\s*\{.*?\})', code, re.DOTALL)
    return functions

def missing_bounds_check(code):
    functions = split_into_functions(code)
    risky_keywords = ['recv', 'read', 'strcpy', 'memcpy', 'gets', 'strcat', 'write']
    for func in functions:
        found_risky = any(kw in func for kw in risky_keywords)
        has_check = any('if' in line and ('<' in line or '>' in line or '<=' in line or '>=' in line) for line in func.splitlines())
        if found_risky and not has_check:
            return 1
    return 0

def analyze_risks(code):
    risk_flags = {
        "Missing Null Check": missing_null_check_func(code),
        "Race Condition": detect_race_condition(code),
        "Missing Bounds Check": missing_bounds_check(code),
        "Unsafe Memory Allocation": detect_missing_malloc_check(code),
        "Logging Without Halting": logging_but_no_blocking(code),
        "issue_detected": 0
    }
    if any(v != 0 for v in risk_flags.values() if v is not False):
        risk_flags["issue_detected"] = 1
    return risk_flags

# Load and preprocess data
train_path = '/home/phoenix/Desktop/TA/Re_ New Code/Linux-Data-Updated.csv'
test_paths = {
    'Android': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Android-Data.csv',
    'Chrome': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Chrome-Data.csv',
    'ImageMagick': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/ImageMagick-Data.csv'
}

def load_or_generate_embeddings(df, path, filename):
    embed_file = f"{filename}_embeddings.npy"
    heur_file = f"{filename}_heuristics.npy"
    commit_file = f"{filename}_commit_ids.npy"

    if os.path.exists(embed_file) and os.path.exists(heur_file) and os.path.exists(commit_file):
        embeddings = np.load(embed_file)
        heuristics = np.load(heur_file)
        commit_ids = np.load(commit_file)
    else:
        embeddings = []
        heuristics = []
        commit_ids = []
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing GraphCodeBERT embeddings and heuristics ({filename})"):
            code = row['PCode']
            embedding = get_code_features(code)
            risk_flags = analyze_risks(code)
            embeddings.append(embedding)
            heuristics.append([risk_flags[h] for h in [
                "Missing Null Check", "Race Condition", "Missing Bounds Check",
                "Unsafe Memory Allocation", "Logging Without Halting", "issue_detected"
            ]])
            commit_ids.append(row['Commit_Id'])
        embeddings = np.array(embeddings)
        heuristics = np.array(heuristics)
        np.save(embed_file, embeddings)
        np.save(heur_file, heuristics)
        np.save(commit_file, commit_ids)
    return embeddings, heuristics, commit_ids

df_train = pd.read_csv(train_path)
df_train['PCode'] = df_train['PCode'].astype(str)
embeddings_train, heuristics_train, commit_ids_train = load_or_generate_embeddings(df_train, train_path, "train")

# Combine embeddings and heuristics for training and split for validation
features_train = np.concatenate([embeddings_train, heuristics_train], axis=1)
train_data, val_data = train_test_split(features_train, test_size=0.2, random_state=42)

# Hyperparameter tuning for K-means
param_grid = {
    'n_clusters': range(2, 5),  # Test 2 to 10 clusters
    'n_init': [10, 20, 50],     # Number of initializations
    'tol': [1e-4, 1e-5]         # Convergence tolerance
}
results = []
for params in tqdm(product(*param_grid.values()), desc="Tuning K-means"):
    n_clusters, n_init, tol = params
    clustering = KMeans(n_clusters=n_clusters, init='k-means++', n_init=n_init, tol=tol, random_state=42)
    train_clusters = clustering.fit_predict(features_train)
    if len(np.unique(train_clusters)) > 1:  # Ensure at least 2 clusters
        silhouette = silhouette_score(features_train, train_clusters)
        ch_score = calinski_harabasz_score(features_train, train_clusters)
        db_score = davies_bouldin_score(features_train, train_clusters)
        results.append({
            'n_clusters': n_clusters,
            'n_init': n_init,
            'tol': tol,
            'silhouette': silhouette,
            'ch_score': ch_score,
            'db_score': db_score
        })
        print(f"n_clusters={n_clusters}, n_init={n_init}, tol={tol}, Silhouette: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

# Select best configuration based on silhouette score
best_result = max(results, key=lambda x: x['silhouette'])
print(f"Best Configuration: n_clusters={best_result['n_clusters']}, n_init={best_result['n_init']}, tol={best_result['tol']}, "
      f"Silhouette: {best_result['silhouette']:.4f}, CH: {best_result['ch_score']:.2f}, DB: {best_result['db_score']:.4f}")
best_params = {
    'n_clusters': best_result['n_clusters'],
    'n_init': best_result['n_init'],
    'tol': best_result['tol']
}

# Process test datasets with initial embeddings and heuristics
test_data = {}
for name, path in test_paths.items():
    df_test = pd.read_csv(path)
    df_test['PCode'] = df_test['PCode'].astype(str)
    embeddings_test, heuristics_test, commit_ids_test = load_or_generate_embeddings(df_test, path, name)
    features_test = np.concatenate([embeddings_test, heuristics_test], axis=1)
    test_data[name] = {
        'features': features_test,
        'heuristics': heuristics_test,
        'commit_ids': np.array(commit_ids_test),
        'codes': df_test['PCode'].values
    }

# Apply tuned K-means clustering to test datasets and analyze
heuristic_names = [
    "Missing Null Check", "Race Condition", "Missing Bounds Check",
    "Unsafe Memory Allocation", "Logging Without Halting", "None"
]
for name in test_data:
    features = test_data[name]['features']
    heuristics = test_data[name]['heuristics']
    commit_ids = test_data[name]['commit_ids']
    codes = test_data[name]['codes']

    clustering = KMeans(n_clusters=best_params['n_clusters'], init='k-means++', n_init=best_params['n_init'],
                        tol=best_params['tol'], random_state=42)
    clusters = clustering.fit_predict(features)

    # Save to dictionary
    test_data[name].update({
        'clusters': clusters
    })

    # Cluster evaluation metrics
    silhouette = silhouette_score(features, clusters)
    ch_score = calinski_harabasz_score(features, clusters)
    db_score = davies_bouldin_score(features, clusters)
    print(f"\n[{name}] Clustering Metrics:")
    print(f"Silhouette Score: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

    # Top anomalies by cluster size
    print(f"Top 5 Largest Clusters in {name}:")
    unique, counts = np.unique(clusters, return_counts=True)
    top_clusters = np.argsort(-counts)[:5]
    for cluster_idx in top_clusters:
        cluster_size = counts[cluster_idx]
        cluster_indices = np.where(clusters == unique[cluster_idx])[0]
        sample_commit = commit_ids[cluster_indices[0]] if len(cluster_indices) > 0 else "N/A"
        print(f"Cluster {unique[cluster_idx]}: Size = {cluster_size}, Sample Commit ID = {sample_commit}")

    # Save output dataframe
    df_out = pd.DataFrame({
        'Commit_ID': commit_ids,
        'Cluster': clusters,
        'Heuristic_Label': [heuristic_names[np.argmax(h) if np.max(h) > 0 else -1] for h in heuristics],
        'Code': codes
    })
    df_out.to_csv(f"{name}_cluster_results.csv", index=False)

    # Cluster summary with full heuristic counts
    print(f"\nHeuristic distribution by cluster in {name}:")
    cluster_heuristic_counts = []
    for c in range(best_params['n_clusters']):
        idx = np.where(clusters == c)[0]
        hsum = np.sum(heuristics[idx], axis=0)
        hsum[-1] = np.sum(heuristics[idx, -1] == 0)  # Correct 'None' count
        cluster_heuristic_counts.append(hsum)
        top = np.argmax(hsum[:-1])  # Index of dominant heuristic (excluding 'None')
        dominant_heuristic = heuristic_names[top]
        dominant_count = hsum[top]
        print(f"\nCluster {c}:")
        print(f"Dominant Heuristic = {dominant_heuristic}, Count = {dominant_count}")
        for h_idx, h_name in enumerate(heuristic_names):
            print(f"{h_name}: {hsum[h_idx]}")

    # Detect and print false positives for "None" heuristic
    print(f"\n{name} Dataset - False Positives for 'None' Heuristic:")
    false_positives = []
    for idx in range(len(heuristics)):
        if heuristics[idx, -1] == 1:  # Sample labeled as "None"
            if np.any(heuristics[idx, :-1] == 1):  # Check if any other heuristic is true
                false_positive_heuristics = [h for h_idx, h in enumerate(heuristic_names[:-1]) if heuristics[idx, h_idx] == 1]
                false_positives.append((idx, false_positive_heuristics))

    if false_positives:
        print(f"Found {len(false_positives)} false positive(s) where 'None' was incorrectly assigned:")
        for idx, conflicting_heuristics in false_positives:
            print(f"\nCommit ID: {commit_ids[idx]}")
            print(f"Conflicting Heuristics: {', '.join(conflicting_heuristics)}")
            print(f"Code Snippet: {codes[idx][:200]}...\n")
    else:
        print("No false positives found for 'None' heuristic.")

    # t-SNE plot with cluster markers, boundaries, and side labels
    tsne = TSNE(n_components=2, random_state=42)
    tsne_coords = tsne.fit_transform(features)
    tsne_df = pd.DataFrame(tsne_coords, columns=['TSNE1', 'TSNE2'])
    tsne_df['Cluster'] = clusters
    tsne_df['Heuristic'] = [heuristic_names[np.argmax(h) if np.max(h) > 0 else -1] for h in heuristics]

    plt.figure(figsize=(14, 8))
    scatter = sns.scatterplot(data=tsne_df, x='TSNE1', y='TSNE2', hue='Heuristic', style='Cluster',
                              palette={
                                  'Missing Null Check': 'green',
                                  'Race Condition': 'red',
                                  'Missing Bounds Check': 'blue',
                                  'Unsafe Memory Allocation': 'purple',
                                  'Logging Without Halting': 'orange',
                                  'None': 'gray'
                              }, marker='o', s=100)
    plt.title(f"t-SNE of {name} - GraphCodeBERT + Heuristics + Tuned K-means")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend(title=' ', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Compute and plot convex hulls for cluster boundaries
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 1:
            cluster_points = tsne_coords[cluster_indices]
            hull = ConvexHull(cluster_points)
            for simplex in hull.simplices:
                plt.plot(cluster_points[simplex, 0], cluster_points[simplex, 1], 'k-', alpha=0.5)

    # Label clusters outside the boundaries
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 0:
            cluster_points = tsne_coords[cluster_indices]
            cluster_center = cluster_points.mean(axis=0)
            x_range = tsne_coords[:, 0].max() - tsne_coords[:, 0].min()
            y_range = tsne_coords[:, 1].max() - tsne_coords[:, 1].min()
            offset = 0.1 * max(x_range, y_range)
            outward_x = cluster_center[0] + offset * np.sign(x_range)
            outward_y = cluster_center[1] + offset * np.sign(y_range)
            plt.text(outward_x, outward_y, f'Cluster {cluster}',
                     fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))



In [ ]:
# Dataset and metric setup
datasets = list(test_data.keys())
metrics = ['Silhouette', 'CH', 'DB']
x_locs = np.arange(len(metrics))
bar_width = 0.2
colors = ['tab:blue', 'tab:orange', 'tab:green']

# Step 1: Collect raw scores
raw_scores = []
for name in datasets:
    sil = silhouette_score(test_data[name]['features'], test_data[name]['clusters'])
    ch = calinski_harabasz_score(test_data[name]['features'], test_data[name]['clusters']) / 1000
    db = davies_bouldin_score(test_data[name]['features'], test_data[name]['clusters'])
    raw_scores.append([sil, ch, db])

raw_scores = np.array(raw_scores)  # shape: (num_datasets, num_metrics)

# Step 2: Normalize for visualization
scaler = MinMaxScaler()
normalized_scores = scaler.fit_transform(raw_scores)

# Add offset to avoid invisible bars
normalized_scores += 0.05
normalized_scores = np.clip(normalized_scores, 0, 1)

# Step 3: Plotting
fig, ax = plt.subplots(figsize=(12, 6))  # Increased width to accommodate legend

for i, (name, color) in enumerate(zip(datasets, colors)):
    offset = (i - 1) * bar_width
    bars = ax.bar(x_locs + offset, normalized_scores[i], width=bar_width, label=name, color=color)

    # Add actual score labels above each bar
    for j, bar in enumerate(bars):
        height = bar.get_height()
        actual_score = raw_scores[i][j]
        ax.text(bar.get_x() + bar.get_width() / 2, height + 0.02,
                f"{actual_score:.2f}", ha='center', va='bottom', fontsize=9)

# Step 4: Axes formatting
ax.set_ylabel("Normalized Metric Score (0–1)\n Actual Metric Values Printed Above Each Bar", fontsize=12)
ax.set_xticks(x_locs)
ax.set_xticklabels(metrics, fontsize=12)
ax.set_title("Normalized Clustering Metrics Across Test Datasets (Android, Chrome, ImageMagick) for GraphCodeBERT + Heuristics + Tuned K-means", fontsize=14)
ax.grid(True, axis='y', linestyle='--', alpha=0.4)
ax.legend(title="Datasets", fontsize=10, bbox_to_anchor=(1.15, 1), loc='upper left')  # Adjusted bbox_to_anchor

plt.tight_layout()
plt.savefig("clustering_metrics_improved.png", dpi=300, bbox_inches='tight')  # Added bbox_inches to ensure legend fits
plt.show()

**M2** Model

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import RobertaTokenizer, RobertaModel
import os
from collections import Counter
from scipy.spatial import ConvexHull

# Set reproducibility seed
np.random.seed(42)  # NumPy seed
torch.manual_seed(42)  # PyTorch seed

# GraphCodeBERT Feature Extraction
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base")

def get_code_features(code):
    inputs = tokenizer(code, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Load and preprocess data
train_path = '/home/phoenix/Desktop/TA/Re_ New Code/Linux-Data-Updated.csv'
test_paths = {
    'Android': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Android-Data.csv',
    'Chrome': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Chrome-Data.csv',
    'ImageMagick': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/ImageMagick-Data.csv'
}

def load_or_generate_embeddings(df, path, filename):
    embed_file = f"{filename}_embeddings.npy"
    commit_file = f"{filename}_commit_ids.npy"

    if os.path.exists(embed_file) and os.path.exists(commit_file):
        embeddings = np.load(embed_file)
        commit_ids = np.load(commit_file)
    else:
        embeddings = []
        commit_ids = []
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing GraphCodeBERT embeddings ({filename})"):
            code = row['PCode']
            embedding = get_code_features(code)
            embeddings.append(embedding)
            commit_ids.append(row['Commit_Id'])
        embeddings = np.array(embeddings)
        np.save(embed_file, embeddings)
        np.save(commit_file, commit_ids)
    return embeddings, commit_ids

df_train = pd.read_csv(train_path)
df_train['PCode'] = df_train['PCode'].astype(str)
embeddings_train, commit_ids_train = load_or_generate_embeddings(df_train, train_path, "train")

# Use embeddings for training and split for validation
train_data, val_data = train_test_split(embeddings_train, test_size=0.2, random_state=42)

# Hyperparameter tuning for K-means
param_grid = {
    'n_clusters': range(2, 11),  # Test 2 to 10 clusters
    'n_init': [10, 20, 50],     # Number of initializations
    'tol': [1e-4, 1e-5]         # Convergence tolerance
}
results = []
for params in tqdm(product(*param_grid.values()), desc="Tuning K-means"):
    n_clusters, n_init, tol = params
    clustering = KMeans(n_clusters=n_clusters, init='k-means++', n_init=n_init, tol=tol, random_state=42)
    train_clusters = clustering.fit_predict(train_data)
    if len(np.unique(train_clusters)) > 1:
        silhouette = silhouette_score(train_data, train_clusters)
        ch_score = calinski_harabasz_score(train_data, train_clusters)
        db_score = davies_bouldin_score(train_data, train_clusters)
        results.append({
            'n_clusters': n_clusters,
            'n_init': n_init,
            'tol': tol,
            'silhouette': silhouette,
            'ch_score': ch_score,
            'db_score': db_score
        })
        print(f"n_clusters={n_clusters}, n_init={n_init}, tol={tol}, Silhouette: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

# Select best configuration based on silhouette score
best_result = max(results, key=lambda x: x['silhouette'])
print(f"Best Configuration: n_clusters={best_result['n_clusters']}, n_init={best_result['n_init']}, tol={best_result['tol']}, "
      f"Silhouette: {best_result['silhouette']:.4f}, CH: {best_result['ch_score']:.2f}, DB: {best_result['db_score']:.4f}")
best_params = {
    'n_clusters': best_result['n_clusters'],
    'n_init': best_result['n_init'],
    'tol': best_result['tol']
}

# Process test datasets with embeddings
test_data = {}
for name, path in test_paths.items():
    df_test = pd.read_csv(path)
    df_test['PCode'] = df_test['PCode'].astype(str)
    embeddings_test, commit_ids_test = load_or_generate_embeddings(df_test, path, name)
    test_data[name] = {
        'features': embeddings_test,
        'commit_ids': np.array(commit_ids_test),
        'codes': df_test['PCode'].values
    }

# Apply tuned K-means clustering to test datasets and analyze
for name in test_data:
    features = test_data[name]['features']
    commit_ids = test_data[name]['commit_ids']
    codes = test_data[name]['codes']

    clustering = KMeans(n_clusters=best_params['n_clusters'], init='k-means++', n_init=best_params['n_init'],
                        tol=best_params['tol'], random_state=42)
    clusters = clustering.fit_predict(features)

    # Save to dictionary
    test_data[name].update({
        'clusters': clusters
    })

    # Cluster evaluation metrics
    silhouette = silhouette_score(features, clusters)
    ch_score = calinski_harabasz_score(features, clusters)
    db_score = davies_bouldin_score(features, clusters)
    print(f"\n[{name}] Clustering Metrics:")
    print(f"Silhouette Score: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

    # Top anomalies by cluster size
    print(f"Top 5 Largest Clusters in {name}:")
    unique, counts = np.unique(clusters, return_counts=True)
    top_clusters = np.argsort(-counts)[:5]
    for cluster_idx in top_clusters:
        cluster_size = counts[cluster_idx]
        cluster_indices = np.where(clusters == unique[cluster_idx])[0]
        sample_commit = commit_ids[cluster_indices[0]] if len(cluster_indices) > 0 else "N/A"
        print(f"Cluster {unique[cluster_idx]}: Size = {cluster_size}, Sample Commit ID = {sample_commit}")

    # Save output dataframe
    df_out = pd.DataFrame({
        'Commit_ID': commit_ids,
        'Cluster': clusters,
        'Code': codes
    })
    df_out.to_csv(f"{name}_cluster_results.csv", index=False)

    # t-SNE plot with cluster markers, boundaries, and side labels
    tsne = TSNE(n_components=2, random_state=42)
    tsne_coords = tsne.fit_transform(features)
    tsne_df = pd.DataFrame(tsne_coords, columns=['TSNE1', 'TSNE2'])
    tsne_df['Cluster'] = clusters

    plt.figure(figsize=(14, 8))
    scatter = sns.scatterplot(data=tsne_df, x='TSNE1', y='TSNE2', hue='Cluster', style='Cluster',
                              palette='tab10', marker='o', s=100)
    plt.title(f"t-SNE of {name} - GraphCodeBERT + Tuned K-means")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Compute and plot convex hulls for cluster boundaries with error handling
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 2:
            cluster_points = tsne_coords[cluster_indices]
            if np.linalg.norm(cluster_points.max(axis=0) - cluster_points.min(axis=0)) > 1e-10:
                try:
                    hull = ConvexHull(cluster_points)
                    for simplex in hull.simplices:
                        plt.plot(cluster_points[simplex, 0], cluster_points[simplex, 1], 'k-', alpha=0.5)
                except QhullError as e:
                    print(f"Warning: ConvexHull failed for Cluster {cluster} in {name}: {e}")
            else:
                print(f"Warning: Insufficient variance in Cluster {cluster} in {name}, skipping hull.")

    # Label clusters outside the boundaries
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 0:
            cluster_points = tsne_coords[cluster_indices]
            cluster_center = cluster_points.mean(axis=0)
            x_range = tsne_coords[:, 0].max() - tsne_coords[:, 0].min()
            y_range = tsne_coords[:, 1].max() - tsne_coords[:, 1].min()
            offset = 0.1 * max(x_range, y_range)
            outward_x = cluster_center[0] + offset * np.sign(x_range)
            outward_y = cluster_center[1] + offset * np.sign(y_range)
            plt.text(outward_x, outward_y, f'Cluster {cluster}',
                     fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.savefig(f"{name}_tsne_plot.png")
    plt.show()

# Combined metric plot (corrected to normalized style with legend on right)
datasets = list(test_data.keys())
metrics = ['Silhouette', 'CH', 'DB']
x_locs = np.arange(len(metrics))
bar_width = 0.2
colors = ['tab:blue', 'tab:orange', 'tab:green']

# Collect raw scores
raw_scores = []
for name in datasets:
    sil = silhouette_score(test_data[name]['features'], test_data[name]['clusters'])
    ch = calinski_harabasz_score(test_data[name]['features'], test_data[name]['clusters']) / 1000
    db = davies_bouldin_score(test_data[name]['features'], test_data[name]['clusters'])
    raw_scores.append([sil, ch, db])

raw_scores = np.array(raw_scores)

# Normalize for visualization
scaler = MinMaxScaler()
normalized_scores = scaler.fit_transform(raw_scores)

# Add offset to avoid invisible bars
normalized_scores += 0.05
normalized_scores = np.clip(normalized_scores, 0, 1)

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))  # Increased width for legend

for i, (name, color) in enumerate(zip(datasets, colors)):
    offset = (i - 1) * bar_width
    bars = ax.bar(x_locs + offset, normalized_scores[i], width=bar_width, label=name, color=color)

    # Add actual score labels above each bar
    for j, bar in enumerate(bars):
        height = bar.get_height()
        actual_score = raw_scores[i][j]
        ax.text(bar.get_x() + bar.get_width() / 2, height + 0.02,
                f"{actual_score:.2f}", ha='center', va='bottom', fontsize=9)

# Axes formatting
ax.set_ylabel("Normalized Metric Score (0–1)\n Actual Metric Values Printed Above Each Bar", fontsize=12)
ax.set_xticks(x_locs)
ax.set_xticklabels(metrics, fontsize=12)
ax.set_title("Normalized Clustering Metrics Across Test Datasets (Android, Chrome, ImageMagick) for GraphCodeBERT + Tuned K-means", fontsize=14)
ax.grid(True, axis='y', linestyle='--', alpha=0.4)
ax.legend(title="Datasets", fontsize=10, bbox_to_anchor=(1.15, 1), loc='upper left')  # Legend on right

plt.tight_layout()
plt.savefig("clustering_metrics_improved.png", dpi=300, bbox_inches='tight')
plt.show()

**M1** Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from itertools import product
import os
from collections import Counter
from scipy.spatial import ConvexHull, QhullError  # Added QhullError import
from sklearn.preprocessing import MinMaxScaler

# Set reproducibility seed
np.random.seed(42)  # NumPy seed

# Heuristic Detection Functions
def detect_missing_malloc_check(code: str) -> int:
    alloc_pattern = r'\b(\w+)\s*\*\s*(\w+)\s*=\s*\([^\)]*\)\s*(malloc|calloc|realloc)\s*\([^;]+?\);|\b(\w+)\s*\*\s*(\w+)\s*=\s*(malloc|calloc|realloc)\s*\([^;]+?\);'
    matches = re.findall(alloc_pattern, code)
    ptrs = [match[1] if match[1] else match[4] for match in matches if match[1] or match[4]]
    for ptr in ptrs:
        is_checked = False
        is_used = False
        null_check_patterns = [rf'if\s*\(\s*{ptr}\s*\)', rf'if\s*\(\s*{ptr}\s*!=\s*NULL\s*\)', rf'if\s*\(\s*!{ptr}\s*\)', rf'if\s*\(\s*NULL\s*!=\s*{ptr}\s*\)']
        usage_patterns = [rf'{ptr}\s*\[', rf'\*\s*{ptr}', rf'{ptr}\s*->', rf'{ptr}\s*\(', rf'{ptr}\s*=']
        for pat in null_check_patterns:
            if re.search(pat, code):
                is_checked = True
                break
        for pat in usage_patterns:
            if re.search(pat, code):
                is_used = True
                break
        if is_used and not is_checked:
            return 1
    return 0

def missing_null_check_func(code):
    functions = re.findall(r'\b\w+\s+\w+\s*\([^)]*\)\s*\{[^{}]*\}', code, re.DOTALL)
    for func in functions:
        ptr_decl = re.findall(r'\b\w+\s*\*\s*(\w+)\s*=\s*[^;]+;', func)
        ptr_func_decl = re.findall(r'\b\w+\s*\*\s*(\w+)\s*;', func)
        ptr_vars = list(set(ptr_decl + ptr_func_decl))
        for ptr in ptr_vars:
            unsafe_use = re.search(rf'[^a-zA-Z0-9_]({ptr})(->|\[\d*]|[^\w]?\*)|\b{ptr}\s*\(', func)
            if not unsafe_use:
                continue
            null_check = re.search(rf'if\s*\(\s*(!\s*{ptr}|{ptr}\s*==\s*NULL|{ptr}\s*!=\s*NULL)\s*\)', func)
            if not null_check:
                return 1
    return 0

def detect_race_condition(code: str) -> bool:
    field_assignment_pattern = r'\b\w+\s*(->|\.)\s*\w+\s*=.*?;'
    control_block_pattern = r'\b(if|while|for|switch)\s*\([^\)]+\)\s*{[^}]*' + field_assignment_pattern + r'[^}]*}'
    matches_control_blocks = re.findall(control_block_pattern, code, re.DOTALL)
    locking_primitive_pattern = r'\b(mutex_lock|pthread_mutex_lock|spin_lock)\b'
    unlocking_primitive_pattern = r'\b(mutex_unlock|pthread_mutex_unlock|spin_unlock)\b'
    has_locking = re.search(locking_primitive_pattern, code)
    has_unlocking = re.search(unlocking_primitive_pattern, code)
    if matches_control_blocks and not (has_locking and has_unlocking):
        return True
    return False

def logging_but_no_blocking(code):
    log_lines = re.findall(r'(syslog\([^)]*\)|printk\([^)]*\))', code)
    for line in log_lines:
        log_index = code.find(line)
        snippet = code[log_index:log_index+150]
        if not re.search(r'(return|exit|break|continue)', snippet):
            return 1
    return 0

def split_into_functions(code):
    functions = re.findall(r'([a-zA-Z_][a-zA-Z0-9_\*\s]*\s+[a-zA-Z_][a-zA-Z0-9_]*\s*\(.*?\)\s*\{.*?\})', code, re.DOTALL)
    return functions

def missing_bounds_check(code):
    functions = split_into_functions(code)
    risky_keywords = ['recv', 'read', 'strcpy', 'memcpy', 'gets', 'strcat', 'write']
    for func in functions:
        found_risky = any(kw in func for kw in risky_keywords)
        has_check = any('if' in line and ('<' in line or '>' in line or '<=' in line or '>=' in line) for line in func.splitlines())
        if found_risky and not has_check:
            return 1
    return 0

def analyze_risks(code):
    risk_flags = {
        "Missing Null Check": missing_null_check_func(code),
        "Race Condition": detect_race_condition(code),
        "Missing Bounds Check": missing_bounds_check(code),
        "Unsafe Memory Allocation": detect_missing_malloc_check(code),
        "Logging Without Halting": logging_but_no_blocking(code),
        "issue_detected": 0
    }
    if any(v != 0 for v in risk_flags.values() if v is not False):
        risk_flags["issue_detected"] = 1
    return risk_flags

# Load and preprocess data
train_path = '/home/phoenix/Desktop/TA/Re_ New Code/Linux-Data-Updated.csv'
test_paths = {
    'Android': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Android-Data.csv',
    'Chrome': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/Chrome-Data.csv',
    'ImageMagick': '/home/phoenix/Desktop/TA/Re_ New Code/CSV Data of three testing projects/ImageMagick-Data.csv'
}

def load_or_generate_heuristics(df, path, filename):
    heur_file = f"{filename}_heuristics.npy"
    commit_file = f"{filename}_commit_ids.npy"

    if os.path.exists(heur_file) and os.path.exists(commit_file):
        heuristics = np.load(heur_file)
        commit_ids = np.load(commit_file)
    else:
        heuristics = []
        commit_ids = []
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing heuristics ({filename})"):
            code = row['PCode']
            risk_flags = analyze_risks(code)
            heuristics.append([risk_flags[h] for h in [
                "Missing Null Check", "Race Condition", "Missing Bounds Check",
                "Unsafe Memory Allocation", "Logging Without Halting", "issue_detected"
            ]])
            commit_ids.append(row['Commit_Id'])
        heuristics = np.array(heuristics)
        np.save(heur_file, heuristics)
        np.save(commit_file, commit_ids)
    return heuristics, commit_ids

df_train = pd.read_csv(train_path)
df_train['PCode'] = df_train['PCode'].astype(str)
heuristics_train, commit_ids_train = load_or_generate_heuristics(df_train, train_path, "train")

# Use heuristics for training and split for validation
train_data, val_data = train_test_split(heuristics_train, test_size=0.2, random_state=42)

# Hyperparameter tuning for K-means
param_grid = {
    'n_clusters': range(2, 7),  # Test 2 to 10 clusters
    'n_init': [10, 20, 50],     # Number of initializations
    'tol': [1e-4, 1e-5]         # Convergence tolerance
}
results = []
for params in tqdm(product(*param_grid.values()), desc="Tuning K-means"):
    n_clusters, n_init, tol = params
    clustering = KMeans(n_clusters=n_clusters, init='k-means++', n_init=n_init, tol=tol, random_state=42)
    train_clusters = clustering.fit_predict(train_data)
    if len(np.unique(train_clusters)) > 1:
        silhouette = silhouette_score(train_data, train_clusters)
        ch_score = calinski_harabasz_score(train_data, train_clusters)
        db_score = davies_bouldin_score(train_data, train_clusters)
        results.append({
            'n_clusters': n_clusters,
            'n_init': n_init,
            'tol': tol,
            'silhouette': silhouette,
            'ch_score': ch_score,
            'db_score': db_score
        })
        print(f"n_clusters={n_clusters}, n_init={n_init}, tol={tol}, Silhouette: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

# Select best configuration based on silhouette score
best_result = max(results, key=lambda x: x['silhouette'])
print(f"Best Configuration: n_clusters={best_result['n_clusters']}, n_init={best_result['n_init']}, tol={best_result['tol']}, "
      f"Silhouette: {best_result['silhouette']:.4f}, CH: {best_result['ch_score']:.2f}, DB: {best_result['db_score']:.4f}")
best_params = {
    'n_clusters': best_result['n_clusters'],
    'n_init': best_result['n_init'],
    'tol': best_result['tol']
}

# Process test datasets with heuristics
test_data = {}
for name, path in test_paths.items():
    df_test = pd.read_csv(path)
    df_test['PCode'] = df_test['PCode'].astype(str)
    heuristics_test, commit_ids_test = load_or_generate_heuristics(df_test, path, name)
    test_data[name] = {
        'features': heuristics_test,
        'heuristics': heuristics_test,
        'commit_ids': np.array(commit_ids_test),
        'codes': df_test['PCode'].values
    }

# Apply tuned K-means clustering to test datasets and analyze
heuristic_names = [
    "Missing Null Check", "Race Condition", "Missing Bounds Check",
    "Unsafe Memory Allocation", "Logging Without Halting", "None"
]
for name in test_data:
    features = test_data[name]['features']
    heuristics = test_data[name]['heuristics']
    commit_ids = test_data[name]['commit_ids']
    codes = test_data[name]['codes']

    clustering = KMeans(n_clusters=best_params['n_clusters'], init='k-means++', n_init=best_params['n_init'],
                        tol=best_params['tol'], random_state=42)
    clusters = clustering.fit_predict(features)

    # Save to dictionary
    test_data[name].update({
        'clusters': clusters
    })

    # Cluster evaluation metrics
    silhouette = silhouette_score(features, clusters)
    ch_score = calinski_harabasz_score(features, clusters)
    db_score = davies_bouldin_score(features, clusters)
    print(f"\n[{name}] Clustering Metrics:")
    print(f"Silhouette Score: {silhouette:.4f}, CH: {ch_score:.2f}, DB: {db_score:.4f}")

    # Top anomalies by cluster size
    print(f"Top 5 Largest Clusters in {name}:")
    unique, counts = np.unique(clusters, return_counts=True)
    top_clusters = np.argsort(-counts)[:5]
    for cluster_idx in top_clusters:
        cluster_size = counts[cluster_idx]
        cluster_indices = np.where(clusters == unique[cluster_idx])[0]
        sample_commit = commit_ids[cluster_indices[0]] if len(cluster_indices) > 0 else "N/A"
        print(f"Cluster {unique[cluster_idx]}: Size = {cluster_size}, Sample Commit ID = {sample_commit}")

    # Save output dataframe
    df_out = pd.DataFrame({
        'Commit_ID': commit_ids,
        'Cluster': clusters,
        'Heuristic_Label': [heuristic_names[np.argmax(h) if np.max(h) > 0 else -1] for h in heuristics],
        'Code': codes
    })
    df_out.to_csv(f"{name}_cluster_results.csv", index=False)

    # Cluster summary with heuristic distribution
    print(f"\nHeuristic distribution by cluster in {name}:")
    cluster_heuristic_counts = []
    for c in range(best_params['n_clusters']):
        idx = np.where(clusters == c)[0]
        hsum = np.sum(heuristics[idx], axis=0)
        hsum[-1] = np.sum(heuristics[idx, -1] == 0)  # Correct 'None' count
        cluster_heuristic_counts.append(hsum)
        top = np.argmax(hsum[:-1])  # Index of dominant heuristic (excluding 'None')
        dominant_heuristic = heuristic_names[top]
        dominant_count = hsum[top]
        print(f"\nCluster {c}:")
        print(f"Dominant Heuristic = {dominant_heuristic}, Count = {dominant_count}")
        for h_idx, h_name in enumerate(heuristic_names):
            print(f"{h_name}: {hsum[h_idx]}")

    # Detect and print false positives for "None" heuristic
    print(f"\n{name} Dataset - False Positives for 'None' Heuristic:")
    false_positives = []
    for idx in range(len(heuristics)):
        if heuristics[idx, -1] == 1:  # Sample labeled as "None"
            if np.any(heuristics[idx, :-1] == 1):  # Check if any other heuristic is true
                false_positive_heuristics = [h for h_idx, h in enumerate(heuristic_names[:-1]) if heuristics[idx, h_idx] == 1]
                false_positives.append((idx, false_positive_heuristics))

    if false_positives:
        print(f"Found {len(false_positives)} false positive(s) where 'None' was incorrectly assigned:")
        for idx, conflicting_heuristics in false_positives:
            print(f"\nCommit ID: {commit_ids[idx]}")
            print(f"Conflicting Heuristics: {', '.join(conflicting_heuristics)}")
            print(f"Code Snippet: {codes[idx][:200]}...\n")
    else:
        print("No false positives found for 'None' heuristic.")

    # t-SNE plot with cluster markers, boundaries, and side labels
    tsne = TSNE(n_components=2, random_state=42)
    tsne_coords = tsne.fit_transform(features)
    tsne_df = pd.DataFrame(tsne_coords, columns=['TSNE1', 'TSNE2'])
    tsne_df['Cluster'] = clusters
    tsne_df['Heuristic'] = [heuristic_names[np.argmax(h) if np.max(h) > 0 else -1] for h in heuristics]

    plt.figure(figsize=(14, 8))
    scatter = sns.scatterplot(data=tsne_df, x='TSNE1', y='TSNE2', hue='Heuristic', style='Cluster',
                              palette={
                                  'Missing Null Check': 'green',
                                  'Race Condition': 'red',
                                  'Missing Bounds Check': 'blue',
                                  'Unsafe Memory Allocation': 'purple',
                                  'Logging Without Halting': 'orange',
                                  'None': 'gray'
                              }, marker='o', s=100)
    plt.title(f"t-SNE of {name} - Heuristics + Tuned K-means")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.legend(title=' ', bbox_to_anchor=(1.05, 1), loc='upper left')

    # Compute and plot convex hulls for cluster boundaries with error handling
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 2:  # Require at least 3 points for a 2D hull
            cluster_points = tsne_coords[cluster_indices]
            # Check for sufficient variance to avoid collinearity
            if np.linalg.norm(cluster_points.max(axis=0) - cluster_points.min(axis=0)) > 1e-10:
                try:
                    hull = ConvexHull(cluster_points)
                    for simplex in hull.simplices:
                        plt.plot(cluster_points[simplex, 0], cluster_points[simplex, 1], 'k-', alpha=0.5)
                except QhullError as e:
                    print(f"Warning: ConvexHull failed for Cluster {cluster} in {name} due to coplanarity or low dimensionality: {e}")
            else:
                print(f"Warning: Insufficient variance in Cluster {cluster} in {name}, skipping hull.")

    # Label clusters outside the boundaries
    for cluster in range(best_params['n_clusters']):
        cluster_indices = np.where(clusters == cluster)[0]
        if len(cluster_indices) > 0:
            cluster_points = tsne_coords[cluster_indices]
            cluster_center = cluster_points.mean(axis=0)
            x_range = tsne_coords[:, 0].max() - tsne_coords[:, 0].min()
            y_range = tsne_coords[:, 1].max() - tsne_coords[:, 1].min()
            offset = 0.1 * max(x_range, y_range)
            outward_x = cluster_center[0] + offset * np.sign(x_range)
            outward_y = cluster_center[1] + offset * np.sign(y_range)
            plt.text(outward_x, outward_y, f'Cluster {cluster}',
                     fontsize=12, ha='center', va='center', bbox=dict(facecolor='white', alpha=0.8))



# Combined metric plot (corrected to normalized style with legend on right)
datasets = list(test_data.keys())
metrics = ['Silhouette', 'CH', 'DB']
x_locs = np.arange(len(metrics))
bar_width = 0.2
colors = ['tab:blue', 'tab:orange', 'tab:green']

# Collect raw scores
raw_scores = []
for name in datasets:
    sil = silhouette_score(test_data[name]['features'], test_data[name]['clusters'])
    ch = calinski_harabasz_score(test_data[name]['features'], test_data[name]['clusters']) / 1000
    db = davies_bouldin_score(test_data[name]['features'], test_data[name]['clusters'])
    raw_scores.append([sil, ch, db])

raw_scores = np.array(raw_scores)

# Normalize for visualization
scaler = MinMaxScaler()
normalized_scores = scaler.fit_transform(raw_scores)

# Add offset to avoid invisible bars
normalized_scores += 0.05
normalized_scores = np.clip(normalized_scores, 0, 1)

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))  # Increased width for legend

for i, (name, color) in enumerate(zip(datasets, colors)):
    offset = (i - 1) * bar_width
    bars = ax.bar(x_locs + offset, normalized_scores[i], width=bar_width, label=name, color=color)

    # Add actual score labels above each bar
    for j, bar in enumerate(bars):
        height = bar.get_height()
        actual_score = raw_scores[i][j]
        ax.text(bar.get_x() + bar.get_width() / 2, height + 0.02,
                f"{actual_score:.2f}", ha='center', va='bottom', fontsize=9)

# Axes formatting
ax.set_ylabel("Normalized Metric Score (0–1)\n Actual Metric Values Printed Above Each Bar", fontsize=12)
ax.set_xticks(x_locs)
ax.set_xticklabels(metrics, fontsize=12)
ax.set_title("Normalized Clustering Metrics Across Test Datasets (Android, Chrome, ImageMagick) for Heuristics + Tuned K-means", fontsize=14)
ax.grid(True, axis='y', linestyle='--', alpha=0.4)
ax.legend(title="Datasets", fontsize=10, bbox_to_anchor=(1.15, 1), loc='upper left')  # Legend on right

plt.tight_layout()
plt.savefig("clustering_metrics_improved.png", dpi=300, bbox_inches='tight')
plt.show()